# ICASSP Experiments

Datasets : [OU-ISIR]

**Data Variations**
* Features = [Accelerometer Only, Accelerometer + Gyroscope]
* Sequence Length = [1, 2, 3, 4]
* Overlap = [no overlap, 0.25, 0.5, 0.75]
* Train-Validation-Test split = [0.8, 0.1, 0.1]

**Model Variations**
* Dropout = [Yes, No]
* BN = [Yes, No]
* Weight Decay = [Yes, No]
* Weight Decay + BN
* Weight Decay + Dropout

**Report:**
* Mean EER
* Std EER
* Min EER
* Max EER

**Visualization**
* Distance Vectors of Best and Worst combinations of all datasets stored in 200 ppt.

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
!cp -r "/content/drive/My Drive/icmla2020/src" /content/
!mv src/datasets.py datasets.py
!mv src/models.py models.py
!mv src/analyze.py analyze.py
!ls

analyze.py  datasets.py  drive	models.py  sample_data	src


In [17]:
data_path = '../data/osaka'
results_path = '../results/'

In [18]:
import datasets, models, analyze
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import activations
# from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
# import nevergrad as ng
np.set_printoptions(precision=4)

In [19]:
def prepare_data(train_sps, val_sps, test_sps, max_cls, split, data, labels, indices_list, run_n):
    train_split = int(split[0] * max_cls)
    val_split = int(split[2] * max_cls)

    rand_indices = np.random.choice(range(max_cls), size=max_cls, replace=False)
    indices_list.append(rand_indices)
    print('Run #%d:'%run_n)
    
    all_data_rand, all_labels_rand = [data[x] for x in rand_indices], [labels[x] for x in rand_indices]
    b_data, b_labels = datasets.generate_batch(train_sps, all_data_rand[:train_split], all_labels_rand[:train_split])
    
    val_data, val_labels = datasets.generate_batch(val_sps, all_data_rand[train_split:train_split+val_split], all_labels_rand[train_split:train_split+val_split])
    ridx = np.random.choice(range(b_data[0].shape[0]), size=b_data[0].shape[0], replace=False)
    
    b_data_test, b_labels_test = datasets.generate_batch(test_sps, all_data_rand[train_split+val_split:], all_labels_rand[train_split+val_split:])

    l_input = b_data[0][ridx]
    r_input = b_data[1][ridx]
    b_labels = b_labels[ridx]

    # print(l_input[0].shape)

    l_input_val = val_data[0]
    r_input_val = val_data[1]

    l_input_test = b_data_test[0]
    r_input_test = b_data_test[1]
    return l_input, r_input, b_labels, l_input_val, r_input_val, val_labels, l_input_test, r_input_test, b_labels_test

In [20]:
def load_dataset(segLen, overlap, acc_only):
    data, labels = datasets.load_segment_osaka(data_path, 
                                            [0,744], 
                                            sample_rate=100,
                                            acc_only=acc_only,
                                            segment_time=segLen, 
                                            overlapped=overlap, 
                                            overlap=overlap, 
                                            downsample=True)
    return data, labels

In [21]:
def build_model1(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    if reg:
        model.add(Conv1D(16, 3, strides=1, input_shape=input_shape, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(16, 3, strides=1, input_shape=input_shape))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(MaxPooling1D(pool_size=2))
    if reg:
        model.add(Conv1D(64, 5, strides=2, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(64, 5, strides=2))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(AveragePooling1D(pool_size=2))
    model.add(GlobalMaxPooling1D())
    return model

In [22]:
def build_model2(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    if reg:
        model.add(Conv1D(16, 3, strides=1, activation='tanh', input_shape=input_shape, kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
        model.add(Conv1D(32, 3, strides=2, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(16, 3, strides=1, activation='tanh', input_shape=input_shape))
        model.add(Conv1D(32, 3, strides=2, activation='relu'))
    
    if bn:
        model.add(BatchNormalization())

    model.add(MaxPooling1D(pool_size=2))

    if reg:
        model.add(Conv1D(64, 5, strides=2, activation='tanh', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
        model.add(Conv1D(128, 5, strides=3, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(64, 5, strides=2, activation='tanh'))
        model.add(Conv1D(128, 5, strides=3, activation='relu'))
    
    if bn:
        model.add(BatchNormalization())

    model.add(GlobalMaxPooling1D())
    return model

In [23]:
def build_model3(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    model.add(Conv1D(64, 5, strides=3, padding='valid', activation='relu', input_shape=input_shape))
    if bn: model.add(BatchNormalization())
    model.add(Conv1D(128, 3, strides=2, padding='valid', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(128, 2, strides=1, padding='valid', activation='tanh'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(GlobalMaxPooling1D())
    return model

In [24]:
def build_model4(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    if reg:
        model.add(Conv1D(32, 3, strides=1, input_shape=input_shape, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(32, 3, strides=1, input_shape=input_shape))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(MaxPooling1D(pool_size=2))
    if reg:
        model.add(Conv1D(128, 5, strides=2, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(128, 5, strides=2))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(AveragePooling1D(pool_size=2))
    model.add(GlobalMaxPooling1D())
    return model

In [25]:
def build_model5(input_shape, bn, reg):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, \
                                        MaxPooling1D, \
                                        AveragePooling1D, \
                                        Dense, Flatten, Dropout, \
                                        BatchNormalization, \
                                        GlobalMaxPooling1D, \
                                        GlobalAveragePooling1D, \
                                        Activation
    from tensorflow.keras import activations
    import tensorflow.keras.regularizers as regularizers

    model = Sequential()
    if reg:
        model.add(Conv1D(64, 3, strides=1, input_shape=input_shape, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(64, 3, strides=1, input_shape=input_shape))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(MaxPooling1D(pool_size=2))
    if reg:
        model.add(Conv1D(256, 5, strides=2, 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        bias_regularizer=regularizers.l2(1e-4)))
    else:
        model.add(Conv1D(256, 5, strides=2))
    if bn:
        model.add(BatchNormalization())
    model.add(Activation(activations.relu))
    model.add(AveragePooling1D(pool_size=2))
    model.add(GlobalMaxPooling1D())
    return model

In [26]:
acc_only_list    = [False, True]
segLen_list      = [1, 2]
overlap_list     = [0, 0.25, 0.5, 0.75]
# models_list      = [build_model1, build_model2, build_model3]
models_list      = [build_model4, build_model5]
bn_list          = [True, False]
reg_list         = [True, False]

# Configurations
train_sps       = 50
val_sps         = 20
test_sps        = 20
max_cls         = 744
split           = [0.8, 0.1, 0.1]
output_dropout  = 0.1
runs            = 15
np.set_printoptions(precision=4)

In [27]:
data, labels = load_dataset(2, 0.75, acc_only=False)

# Results of this configuration/experiment
EERs = np.array([np.zeros(runs) for _ in range(3*2*2)])

for run_n in range(runs):
    l_input, r_input, b_labels, l_input_val, r_input_val, val_labels, l_input_test, r_input_test, b_labels_test = prepare_data(train_sps,
                                                                                                                                val_sps,
                                                                                                                                test_sps,
                                                                                                                                max_cls,
                                                                                                                                split,
                                                                                                                                data,
                                                                                                                                labels,
                                                                                                                                [],
                                                                                                                                run_n)
    adam = Adam(learning_rate=0.001)
    earlystop = EarlyStopping(monitor='val_accuracy', patience=5)
    for bn in bn_list:
        for reg in reg_list:
            for model_index, build_cnn in enumerate(models_list):
                cnn = build_cnn(l_input[0].shape, bn, reg)
                siamese = models.build_siamese(l_input[0].shape, cnn, output_dropout)
                siamese.compile(loss='binary_crossentropy', optimizer=adam, metrics=[BinaryAccuracy(name='accuracy')])
                if run_n == 0: 
                    cnn.summary()
                hist = siamese.fit([l_input, r_input], 
                                    b_labels, 
                                    shuffle=True,
                                    batch_size=64,
                                    epochs=100,
                                    callbacks=[earlystop],
                                    validation_data=([l_input_val, r_input_val], val_labels),
                                    verbose=2
                                    )
                FRR, FAR, EER, EER_th = analyze.ROC(siamese, [l_input_test, r_input_test], b_labels_test)
                EERs[4 * model_index + 2*bn + reg , run_n] = EER
                np.save(results_path + 'osaka_filters_eers2.npy', EERs)


Run #0:
Model: "sequential_120"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_240 (Conv1D)          (None, 98, 32)            608       
_________________________________________________________________
batch_normalization_120 (Bat (None, 98, 32)            128       
_________________________________________________________________
activation_240 (Activation)  (None, 98, 32)            0         
_________________________________________________________________
max_pooling1d_120 (MaxPoolin (None, 49, 32)            0         
_________________________________________________________________
conv1d_241 (Conv1D)          (None, 23, 128)           20608     
_________________________________________________________________
batch_normalization_121 (Bat (None, 23, 128)           512       
_________________________________________________________________
activation_241 (Activation)  (None, 23, 128)

Train on 59500 samples, validate on 2960 samples
Epoch 1/100
59500/59500 - 13s - loss: 0.1815 - accuracy: 0.9335 - val_loss: 0.1284 - val_accuracy: 0.9537
Epoch 2/100
59500/59500 - 12s - loss: 0.0886 - accuracy: 0.9707 - val_loss: 0.1087 - val_accuracy: 0.9625
Epoch 3/100
59500/59500 - 11s - loss: 0.0615 - accuracy: 0.9811 - val_loss: 0.1151 - val_accuracy: 0.9639
Epoch 4/100
59500/59500 - 12s - loss: 0.0485 - accuracy: 0.9844 - val_loss: 0.1152 - val_accuracy: 0.9581
Epoch 5/100
59500/59500 - 10s - loss: 0.0414 - accuracy: 0.9862 - val_loss: 0.1001 - val_accuracy: 0.9649
Epoch 6/100
59500/59500 - 11s - loss: 0.0353 - accuracy: 0.9890 - val_loss: 0.1024 - val_accuracy: 0.9659
Epoch 7/100
59500/59500 - 10s - loss: 0.0305 - accuracy: 0.9906 - val_loss: 0.1126 - val_accuracy: 0.9659
Epoch 8/100
59500/59500 - 11s - loss: 0.0281 - accuracy: 0.9908 - val_loss: 0.0970 - val_accuracy: 0.9669
Epoch 9/100
59500/59500 - 11s - loss: 0.0249 - accuracy: 0.9915 - val_loss: 0.1071 - val_accuracy: 0.96

Epoch 8/100
59500/59500 - 7s - loss: 0.0919 - accuracy: 0.9897 - val_loss: 0.1732 - val_accuracy: 0.9632
Epoch 9/100
59500/59500 - 7s - loss: 0.0876 - accuracy: 0.9910 - val_loss: 0.1753 - val_accuracy: 0.9628
Epoch 10/100
59500/59500 - 7s - loss: 0.0850 - accuracy: 0.9913 - val_loss: 0.1953 - val_accuracy: 0.9490
Epoch 11/100
59500/59500 - 7s - loss: 0.0830 - accuracy: 0.9916 - val_loss: 0.1832 - val_accuracy: 0.9544
Epoch 12/100
59500/59500 - 7s - loss: 0.0818 - accuracy: 0.9916 - val_loss: 0.1669 - val_accuracy: 0.9520
Model: "sequential_126"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_252 (Conv1D)          (None, 98, 32)            608       
_________________________________________________________________
activation_252 (Activation)  (None, 98, 32)            0         
_________________________________________________________________
max_pooling1d_126 (MaxPoolin (None, 49, 32)         

Epoch 7/100
59500/59500 - 10s - loss: 0.0869 - accuracy: 0.9903 - val_loss: 0.1833 - val_accuracy: 0.9601
Train on 59500 samples, validate on 2960 samples
Epoch 1/100
59500/59500 - 9s - loss: 0.1816 - accuracy: 0.9353 - val_loss: 0.1584 - val_accuracy: 0.9382
Epoch 2/100
59500/59500 - 8s - loss: 0.0880 - accuracy: 0.9721 - val_loss: 0.1417 - val_accuracy: 0.9470
Epoch 3/100
59500/59500 - 8s - loss: 0.0647 - accuracy: 0.9798 - val_loss: 0.1268 - val_accuracy: 0.9537
Epoch 4/100
59500/59500 - 8s - loss: 0.0523 - accuracy: 0.9830 - val_loss: 0.1393 - val_accuracy: 0.9500
Epoch 5/100
59500/59500 - 8s - loss: 0.0428 - accuracy: 0.9863 - val_loss: 0.1323 - val_accuracy: 0.9541
Epoch 6/100
59500/59500 - 8s - loss: 0.0349 - accuracy: 0.9894 - val_loss: 0.1427 - val_accuracy: 0.9534
Epoch 7/100
59500/59500 - 8s - loss: 0.0319 - accuracy: 0.9894 - val_loss: 0.1638 - val_accuracy: 0.9517
Epoch 8/100
59500/59500 - 8s - loss: 0.0273 - accuracy: 0.9914 - val_loss: 0.1579 - val_accuracy: 0.9503
Epoch

Epoch 15/100
59500/59500 - 6s - loss: 0.0221 - accuracy: 0.9929 - val_loss: 0.2534 - val_accuracy: 0.9338
Train on 59500 samples, validate on 2960 samples
Epoch 1/100
59500/59500 - 8s - loss: 0.1814 - accuracy: 0.9304 - val_loss: 0.1822 - val_accuracy: 0.9307
Epoch 2/100
59500/59500 - 7s - loss: 0.0790 - accuracy: 0.9742 - val_loss: 0.1605 - val_accuracy: 0.9439
Epoch 3/100
59500/59500 - 7s - loss: 0.0540 - accuracy: 0.9830 - val_loss: 0.1721 - val_accuracy: 0.9392
Epoch 4/100
59500/59500 - 7s - loss: 0.0408 - accuracy: 0.9871 - val_loss: 0.1751 - val_accuracy: 0.9514
Epoch 5/100
59500/59500 - 7s - loss: 0.0325 - accuracy: 0.9900 - val_loss: 0.1797 - val_accuracy: 0.9527
Epoch 6/100
59500/59500 - 7s - loss: 0.0266 - accuracy: 0.9918 - val_loss: 0.2299 - val_accuracy: 0.9405
Epoch 7/100
59500/59500 - 7s - loss: 0.0221 - accuracy: 0.9933 - val_loss: 0.1740 - val_accuracy: 0.9510
Epoch 8/100
59500/59500 - 7s - loss: 0.0210 - accuracy: 0.9935 - val_loss: 0.1651 - val_accuracy: 0.9544
Epoch

...

Epoch 2/100
59500/59500 - 7s - loss: 0.0822 - accuracy: 0.9730 - val_loss: 0.0946 - val_accuracy: 0.9649
Epoch 3/100
59500/59500 - 8s - loss: 0.0555 - accuracy: 0.9823 - val_loss: 0.0846 - val_accuracy: 0.9666
Epoch 4/100
59500/59500 - 7s - loss: 0.0415 - accuracy: 0.9874 - val_loss: 0.0844 - val_accuracy: 0.9682
Epoch 5/100
59500/59500 - 7s - loss: 0.0328 - accuracy: 0.9897 - val_loss: 0.0907 - val_accuracy: 0.9682
Epoch 6/100
59500/59500 - 7s - loss: 0.0266 - accuracy: 0.9915 - val_loss: 0.0736 - val_accuracy: 0.9726
Epoch 7/100
59500/59500 - 7s - loss: 0.0218 - accuracy: 0.9931 - val_loss: 0.0821 - val_accuracy: 0.9696
Epoch 8/100
59500/59500 - 7s - loss: 0.0179 - accuracy: 0.9939 - val_loss: 0.0835 - val_accuracy: 0.9709
Epoch 9/100
59500/59500 - 8s - loss: 0.0168 - accuracy: 0.9949 - val_loss: 0.0802 - val_accuracy: 0.9723
Epoch 10/100
59500/59500 - 7s - loss: 0.0169 - accuracy: 0.9946 - val_loss: 0.0799 - val_accuracy: 0.9764
Epoch 11/100
59500/59500 - 7s - loss: 0.0139 - accurac

Epoch 3/100
59500/59500 - 8s - loss: 0.1285 - accuracy: 0.9813 - val_loss: 0.2105 - val_accuracy: 0.9473
Epoch 4/100
59500/59500 - 8s - loss: 0.1152 - accuracy: 0.9844 - val_loss: 0.1982 - val_accuracy: 0.9486
Epoch 5/100
59500/59500 - 8s - loss: 0.1082 - accuracy: 0.9863 - val_loss: 0.1898 - val_accuracy: 0.9578
Epoch 6/100
59500/59500 - 8s - loss: 0.1002 - accuracy: 0.9881 - val_loss: 0.1936 - val_accuracy: 0.9588
Epoch 7/100
59500/59500 - 8s - loss: 0.0967 - accuracy: 0.9887 - val_loss: 0.2077 - val_accuracy: 0.9574
Epoch 8/100
59500/59500 - 8s - loss: 0.0914 - accuracy: 0.9902 - val_loss: 0.1937 - val_accuracy: 0.9564
Epoch 9/100
59500/59500 - 8s - loss: 0.0890 - accuracy: 0.9903 - val_loss: 0.2333 - val_accuracy: 0.9422
Epoch 10/100
59500/59500 - 8s - loss: 0.0837 - accuracy: 0.9918 - val_loss: 0.1811 - val_accuracy: 0.9601
Epoch 11/100
59500/59500 - 8s - loss: 0.0817 - accuracy: 0.9920 - val_loss: 0.1890 - val_accuracy: 0.9591
Epoch 12/100
59500/59500 - 8s - loss: 0.0787 - accura

Epoch 11/100
59500/59500 - 11s - loss: 0.0849 - accuracy: 0.9910 - val_loss: 0.1849 - val_accuracy: 0.9649
Epoch 12/100
59500/59500 - 11s - loss: 0.0820 - accuracy: 0.9920 - val_loss: 0.1448 - val_accuracy: 0.9723
Epoch 13/100
59500/59500 - 11s - loss: 0.0832 - accuracy: 0.9909 - val_loss: 0.1529 - val_accuracy: 0.9649
Epoch 14/100
59500/59500 - 11s - loss: 0.0801 - accuracy: 0.9927 - val_loss: 0.1408 - val_accuracy: 0.9693
Epoch 15/100
59500/59500 - 11s - loss: 0.0833 - accuracy: 0.9912 - val_loss: 0.1570 - val_accuracy: 0.9591
Epoch 16/100
59500/59500 - 11s - loss: 0.0818 - accuracy: 0.9921 - val_loss: 0.1431 - val_accuracy: 0.9632
Epoch 17/100
59500/59500 - 11s - loss: 0.0792 - accuracy: 0.9922 - val_loss: 0.1533 - val_accuracy: 0.9672
Train on 59500 samples, validate on 2960 samples
Epoch 1/100
59500/59500 - 10s - loss: 0.1864 - accuracy: 0.9322 - val_loss: 0.1610 - val_accuracy: 0.9402
Epoch 2/100
59500/59500 - 9s - loss: 0.0911 - accuracy: 0.9699 - val_loss: 0.1709 - val_accuracy

Epoch 8/100
59500/59500 - 6s - loss: 0.0441 - accuracy: 0.9855 - val_loss: 0.1478 - val_accuracy: 0.9463
Epoch 9/100
59500/59500 - 6s - loss: 0.0404 - accuracy: 0.9869 - val_loss: 0.1509 - val_accuracy: 0.9486
Epoch 10/100
59500/59500 - 6s - loss: 0.0378 - accuracy: 0.9875 - val_loss: 0.1645 - val_accuracy: 0.9453
Epoch 11/100
59500/59500 - 6s - loss: 0.0340 - accuracy: 0.9893 - val_loss: 0.1454 - val_accuracy: 0.9534
Epoch 12/100
59500/59500 - 6s - loss: 0.0300 - accuracy: 0.9905 - val_loss: 0.1670 - val_accuracy: 0.9449
Epoch 13/100
59500/59500 - 6s - loss: 0.0292 - accuracy: 0.9904 - val_loss: 0.1673 - val_accuracy: 0.9480
Epoch 14/100
59500/59500 - 8s - loss: 0.0275 - accuracy: 0.9910 - val_loss: 0.1510 - val_accuracy: 0.9534
Epoch 15/100
59500/59500 - 6s - loss: 0.0251 - accuracy: 0.9919 - val_loss: 0.1601 - val_accuracy: 0.9392
Epoch 16/100
59500/59500 - 6s - loss: 0.0231 - accuracy: 0.9926 - val_loss: 0.1704 - val_accuracy: 0.9476
Train on 59500 samples, validate on 2960 samples

59500/59500 - 11s - loss: 0.0106 - accuracy: 0.9968 - val_loss: 0.1693 - val_accuracy: 0.9510
Epoch 15/100
59500/59500 - 10s - loss: 0.0090 - accuracy: 0.9969 - val_loss: 0.1659 - val_accuracy: 0.9510
Epoch 16/100
59500/59500 - 10s - loss: 0.0083 - accuracy: 0.9975 - val_loss: 0.1829 - val_accuracy: 0.9547
Epoch 17/100
59500/59500 - 10s - loss: 0.0068 - accuracy: 0.9980 - val_loss: 0.1986 - val_accuracy: 0.9544
Epoch 18/100
59500/59500 - 10s - loss: 0.0085 - accuracy: 0.9973 - val_loss: 0.2046 - val_accuracy: 0.9483
Train on 59500 samples, validate on 2960 samples
Epoch 1/100
59500/59500 - 7s - loss: 0.2469 - accuracy: 0.9149 - val_loss: 0.2465 - val_accuracy: 0.9250
Epoch 2/100
59500/59500 - 7s - loss: 0.1495 - accuracy: 0.9616 - val_loss: 0.2235 - val_accuracy: 0.9402
Epoch 3/100
59500/59500 - 7s - loss: 0.1241 - accuracy: 0.9716 - val_loss: 0.2371 - val_accuracy: 0.9284
Epoch 4/100
59500/59500 - 7s - loss: 0.1099 - accuracy: 0.9776 - val_loss: 0.2115 - val_accuracy: 0.9486
Epoch 5/1

In [28]:
EERs = np.load(results_path + 'osaka_filters_eers2.npy')
EERs[:,15:] = np.load(results_path + 'osaka_filters_eers2.npy')
print(EERs.shape)

ValueError: could not broadcast input array from shape (12,15) into shape (12,0)

In [ ]:
means = np.mean(EERs, axis=1) * 100
print(means)

In [ ]:
stds = np.std(EERs, axis=1) * 100
print(stds)

In [ ]:
mins = np.min(EERs, axis=1) * 100
print(mins)

In [ ]:
mins = np.max(EERs, axis=1) * 100
print(mins)